#### Feedback loop (18th july)
 (GABBR1 C1414904, Baclofen C0004609)

In [8]:
import torch
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import random
import text_util
import openai
from collections import defaultdict
from itertools import cycle
import faiss
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer, DPRContextEncoder, DPRContextEncoderTokenizer

from pathlib import Path
from openai import OpenAI
from agatha.construct.semrep_handler import SemRepHandler
import semantic_search_util
# from semantic_search_util import generate_negatives, st_nodelbl_dict,retrieve_context_path
from agatha.util.sqlite3_lookup import Sqlite3LookupTable
from sentence_transformers import SentenceTransformer, util
import warnings

warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)

In [9]:
agatha_sent_db_path = (
    '/lustre/acslab/shared/Agatha_shared/2021_11_22_full/sentences.sqlite3'
)
sents_db = Sqlite3LookupTable(agatha_sent_db_path)

## 1.Retrieve Path for source and target node from HGCR
Got pair, and retrieve all paths for that pair from Retrieval component

In [12]:
source_cui = 'C1414904' # GABBR1
target_cui = 'C0004609' # Baclofen

In [13]:
df = pd.read_pickle('P1_GABBR1_Baclofen.pkl')
df

,path,score_std,score_mean,dec_path,context_pmids
153,"[C1414904, C1420206, C1426735, C0004609]",0.000000,0.999669,"[GABBR1 gene, SLC6A1 gene, SLC17A8 gene, baclo...","{('C1414904', 'C1420206'): ['26727527'], ('C14..."
134,"[C1414904, C1414925, C0751791, C0004609]",0.000000,0.998868,"[GABBR1 gene, GAD1 gene, Reflex Epilepsy, Audi...","{('C1414904', 'C1414925'): ['25641668'], ('C14..."
27,"[C1414904, C0009595, C0241705, C0004609]",0.000000,0.998843,"[GABBR1 gene, Obsessive-Compulsive Personality...","{('C1414904', 'C0009595'): ['27047397'], ('C00..."
105,"[C1414904, C1367458, C1422223, C0004609]",0.000353,0.998821,"[GABBR1 gene, LGI1 gene, CNTNAP2 gene, baclofen]","{('C1414904', 'C1367458'): ['34836497', '18355..."
165,"[C1414904, C0014553, C1413059, C0004609]",0.000000,0.998617,"[GABBR1 gene, Absence Epilepsy, CACNA1E gene, ...","{('C1414904', 'C0014553'): ['12770685'], ('C00..."
...,...,...,...,...,...
35,"[C1414904, C0393639, C0038220, C0004609]",0.012101,0.811307,"[GABBR1 gene, Hashimoto's encephalitis, Status...","{('C1414904', 'C0393639'): ['34836497'], ('C03..."
166,"[C1414904, C0393639, C0006012, C0004609]",0.000000,0.791663,"[GABBR1 gene, Hashimoto's encephalitis, Border...","{('C1414904', 'C0393639'): ['34836497'], ('C03..."
78,"[C1414904, C0009952, C0080356, C0004609]",0.021340,0.697327,"[GABBR1 gene, Febrile Convulsions, valproate, ...","{('C1414904', 'C0009952'): ['15799783'], ('C00..."
126,"[C1414904, C0270736, C0080356, C0004609]",0.000000,0.662091,"[GABBR1 gene, Essential Tremor, valproate, bac...","{('C1414904', 'C0270736'): ['22321358'], ('C02..."


In [14]:
columns_to_select = ['path', 'context_pmids']
top_20_df = df.loc[:, columns_to_select].head(20)
top_20_df

,path,context_pmids
153,"[C1414904, C1420206, C1426735, C0004609]","{('C1414904', 'C1420206'): ['26727527'], ('C14..."
134,"[C1414904, C1414925, C0751791, C0004609]","{('C1414904', 'C1414925'): ['25641668'], ('C14..."
27,"[C1414904, C0009595, C0241705, C0004609]","{('C1414904', 'C0009595'): ['27047397'], ('C00..."
105,"[C1414904, C1367458, C1422223, C0004609]","{('C1414904', 'C1367458'): ['34836497', '18355..."
165,"[C1414904, C0014553, C1413059, C0004609]","{('C1414904', 'C0014553'): ['12770685'], ('C00..."
72,"[C1414904, C4022738, C0069964, C0004609]","{('C1414904', 'C4022738'): ['36103875'], ('C40..."
50,"[C1414904, C0009595, C0030662, C0004609]","{('C1414904', 'C0009595'): ['27047397'], ('C00..."
132,"[C1414904, C4022738, C1422223, C0004609]","{('C1414904', 'C4022738'): ['36103875'], ('C40..."
81,"[C1414904, C0009952, C1419867, C0004609]","{('C1414904', 'C0009952'): ['15799783'], ('C00..."
204,"[C1414904, C0586323, C0549393, C0004609]","{('C1414904', 'C0586323'): ['16800828'], ('C05..."


In [36]:
pd.set_option('display.max_colwidth', None)

In [6]:
# # get all abstract texts along a specific path as (key: edge(cui,cui) : {(value) {'pmid':'abstract'}})
path_153_context_pmids = df.loc[0]['context_pmids']
path_context_dict = (text_util.get_path_context(path_153_context_pmids,sents_db))
path_context_dict
# pmid_list = [pubid for path in path_context_dict.values() for pubid in path.keys()]
# pmid_list

{('C1414263',
  'C0252643'): {'14506620': 'Attenuation of renal vasopressin V2 receptor upregulation by bosentan, an ETA/ETB receptor antagonist. Circulating endothelin (ET) levels are elevated in heart failure and positively correlated with severity of heart failure. Recent studies demonstrated arginine vasopressin (AVP) V2 mRNA expression was upregulated in the inner medullary collecting duct (IMCD) of cardiomyopathic hamsters (CM). The goal of the present studies was to determine if ET-1 is involved in upregulating the expression of AVP V2 mRNA in the IMCD of CM by using a mixed ETA/ETB receptor antagonist bosentan. Our results showed plasma ET-1 levels increased in CM hamsters and related with the severity of heart failure. The competitive reverse-transcriptase polymerase chain reaction (RT-PCR) method was used to quantify the expression of AVP V2 and aquaporin 2 (AQP2) mRNA in the IMCD. AVP V2 mRNA expression was elevated in placebo-treated CM hamsters and decreased significantly 

### 1.1. PredicatorGraph -> a Subset of Semantic Graph with only UMLS term and Predicates

In [7]:
workingFolder = \
    '/lustre/acslab/shared/Agatha_shared/2021_11_22'

model_path = f'{workingFolder}/model_epoch5_darwin.pt'
embedding_path = f'{workingFolder}/embeddings/predicate_subset'
entity_db = f'{workingFolder}/predicate_entities.sqlite3'
graph_db = f'{workingFolder}/predicate_graph.sqlite3'

In [8]:
model = torch.load(model_path)

model.configure_paths(
  embedding_dir=embedding_path,
  entity_db=entity_db,
  graph_db=graph_db,
)

model = model.eval()

In [50]:
model.predict_from_terms([('m:c1417434', 'm:c0360556')])

[0.22483156621456146]

In [9]:
def extract_pred_cui_pairs(model):
    # nodelist = model.graph.keys() => 'p:c0003128:causes:c1273869','p:c0401187:compared_with:c3522497','p:c0002921:treats(coref):c0745730',
    nodelist = model.graph.keys()
    pred_pairs_list = [p for p in tqdm(nodelist) if p[0] == 'p']
    
    pred_cui_pairs_set = set()
    
    for p in tqdm(pred_pairs_list):
        p_split = p.upper().split(':')
        s = p_split[1]
        o = p_split[-1]
        pred_cui_pairs_set.add(tuple(sorted([s, o])))
    
    return pred_cui_pairs_set

In [10]:
# Replace `model` with your actual model instance containing the graph
pred_cui_pairs_set = extract_pred_cui_pairs(model)
print(len(pred_cui_pairs_set))

100%|██████████| 26535915/26535915 [00:40<00:00, 652418.86it/s]

19086440


### 1.2. SemrepHandler ==> Identify Predicates

In [11]:
nlm_soft_folder = '/lustre/acslab/users/3281/SemRep'
sr_temp_folder = '/lustre/acslab/users/3281/semrep_temp_2024'
sr_replace_utf8_path = '/lustre/acslab/users/3281/SemRep/replace_utf8.jar'

In [12]:
sr_h = SemRepHandler(
    nlm_soft_path=nlm_soft_folder,
    temp_folder=sr_temp_folder,
    replace_utf8_path=sr_replace_utf8_path,
)

Existing services killed.

SKR and WSD services started.


In [13]:
sr_h.sr_binary_path = Path('/lustre/acslab/users/3281/SemRep/public_semrep/bin/semrep.v1.9_2021AB')

In [14]:
sr_h.ProcessList_parallel(['EDNRB and Bosentan'])

Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...


{'s:user_input_0:1': {'sent_text': 'EDNRB and Bosentan ',
  'terms': [{'CID': 'C1414263',
    'pref_name': 'EDNRB gene',
    'extracted_text': 'EDNRB',
    'label': 'UMLS',
    'sem_types': ['gngm'],
    'negated': False},
   {'CID': 'C0252643',
    'pref_name': 'bosentan',
    'extracted_text': 'Bosentan',
    'label': 'UMLS',
    'sem_types': ['orch', 'phsu'],
    'negated': False}],
  'relations': []}}

In [54]:
# pd.DataFrame(sr_h.ProcessList_parallel(['WFS1 and Pioglitazone'])['s:user_input_0:1']['terms'])

## 2. OPENAI LLM Generate Response

In [15]:
llm_model = "gpt-4o"
# swap it with gpt 3.5
open('OAI_CONFIG_LIST').read().strip()
client = OpenAI(
    api_key=open('OAI_CONFIG_LIST').read().strip(),
)

In [16]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Hi, how are you?",
        }
    ],
    model=llm_model,
    temperature=0.5,
)

In [17]:
def oai_get_response(msg, temp, top_p, seed=None):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": msg,
            }
        ],
        model=llm_model,
        temperature=temp,
        top_p=top_p,
    )
    
    reply_text = chat_completion.choices[0].message.content
    
    return reply_text

### 2.1. Retrieve abstracts from pmids per path:

In [31]:
path_153_context_pmids = df.loc[27]['context_pmids']
path_context_dict = (text_util.get_path_context(path_153_context_pmids,sents_db))
# print(path_context_dict)
path_context_dict = (text_util.get_path_context(path_153_context_pmids,sents_db))
path_context_abstracts = semantic_search_util.retrieve_context_path(path_context_dict)
path_context_abstracts

['DNA Methylation at the Neonatal State and at the Time of Diagnosis: Preliminary Support for an Association with the Estrogen Receptor 1, Gamma-Aminobutyric Acid B Receptor 1, and Myelin Oligodendrocyte Glycoprotein in Female Adolescent Patients with OCD. Obsessive-compulsive disorder (OCD) is a neuropsychiatric disorder. Non-genetic factors and their interaction with genes have attracted increasing attention. Epigenetics is regarded an important interface between environmental signals and activation/repression of genomic responses. Epigenetic mechanisms have not previously been examined in OCD in children and adolescents. The aim of the present study was to examine the DNA methylation profile of selected genes in blood spots from neonates later diagnosed with OCD and in the same children/adolescents at the time of diagnosis compared with age- and sex-matched controls. Furthermore, we wanted to characterize the association of the differential methylation profiles with the severity of 

### 2.2. Prompt Engineering

In [21]:
def generate_llm_fix_prompt(source, target, path_context_dict):
    path_context_abstracts = semantic_search_util.retrieve_context_path(path_context_dict)
    llm_prompt_template = 'How would you describe an indirect relationship between {source} and {target} given the following scientific abstracts as context?'
    llm_fix_prompt_str = '\n\n'.join(path_context_abstracts)
    llm_fix_prompt_combo = llm_prompt_template.format(source=source, target=target) + '\n\n' + llm_fix_prompt_str
    
    return llm_fix_prompt_combo

In [18]:
path_context_abstracts = semantic_search_util.retrieve_context_path(path_context_dict)

source_cui = 'EDNRB'  # 'C1421501'
target_cui = 'Bosentan'     # 'C0071097'
llm_prompt_template = 'How would you describe an indirect relationship between {source} and {target} given the following scientific abstracts as context?'

llm_fix_prompt_str = '\n\n'.join(path_context_abstracts)
llm_fix_prompt_combo = llm_prompt_template.format(source=source_cui, target=target_cui) + '\n\n' + llm_fix_prompt_str
llm_fix_prompt_combo

'How would you describe an indirect relationship between EDNRB and Bosentan given the following scientific abstracts as context?\n\nEffect of bosentan (ETA/ETB receptor antagonist) on metabolic changes during stress and diabetes. Elevated plasma ET-1 levels have been reported in several conditions such as stress and diabetes. ET-1 is found to cause insulin resistance and to stimulate liver glycogenolysis. The question arises whether ET-1 has a role in the metabolic changes occurring in such conditions. To test this, we studied the possible effect of the endothelin receptor antagonist, bosentan (50 and 100 mg kg(-1)) on serum glucose and insulin levels as well as on liver glycogen contents in normoglycemic stressed animals. In addition, the effect of bosentan on serum glucose and insulin levels in both mild and severely diabetic rats and its effect on insulin-induced hypoglycemia were also determined. Restraining water immersion stress was used as a model for severe stress reported to e

### 2.3. get resp from LLM as consistance as we can go reduce randomness
NOTE: oai_get_response(msg, temp=1e-19, top_p=1e-9, seed=1234) 
- top_p is a hyperparameter that controls the variety of generated text by only considering tokens with the highest probability

In [19]:
# Note to keep consistant response:
#0.00000001 msg = "What is the capital of France?"
# response = oai_get_response(msg, temp=1e-19, top_p=1e-9, seed=1234)
# print(response)
llm_resp_path_1 = oai_get_response(llm_fix_prompt_combo, temp=1e-19, top_p=1e-9, seed=1234)
llm_resp_path_1

"Based on the provided abstracts, an indirect relationship between EDNRB (Endothelin Receptor Type B) and Bosentan can be described as follows:\n\n**EDNRB and Bosentan: An Indirect Relationship**\n\n**Contextual Overview:**\n1. **Bosentan's Mechanism of Action:**\n   - Bosentan is a dual endothelin receptor antagonist that blocks both ETA and ETB receptors. This blockade affects the physiological actions mediated by endothelin-1 (ET-1), a potent vasoconstrictor involved in various pathological conditions such as stress, diabetes, and hypertension.\n\n2. **Role of EDNRB:**\n   - EDNRB (Endothelin Receptor Type B) is one of the receptors for ET-1. It is involved in vasodilation and clearance of ET-1 from the circulation. The blockade of EDNRB by Bosentan can lead to increased levels of circulating ET-1 and altered vascular responses.\n\n**Indirect Relationship:**\n- **Metabolic Effects:**\n  - In the context of stress and diabetes, Bosentan's blockade of EDNRB (along with ETA) influences

In [65]:
# llm_resp_path_2 = oai_get_response(llm_fix_prompt_combo, temp=1e-19, top_p=1e-9, seed=1234)
# llm_resp_path_2

## 3. Evaluation

In [26]:
with open(
    '/lustre/acslab/shared/Agatha_shared/2021_11_22/2021_11_22_semtypes_to_nodelbl_and_vv_dict.json', 'r'
) as f:
    st_nodelbl_dict = json.load(f)

In [27]:
def generate_negatives(pos_pair, sample_rate=5):
    
    s = pos_pair[0]
    o = pos_pair[1]
    
    if s[0] != 'm':
        s = f'm:{pos_pair[0].lower()}'
    
    if o[0] != 'm':
        o = f'm:{pos_pair[1].lower()}'
    
    if o in st_nodelbl_dict:
        o_st = st_nodelbl_dict[o]
        o_sample_list = random.sample(
            st_nodelbl_dict[o_st],
            sample_rate
        )
    else:
        o_sample_list = []
    
    out_pairs = []
    for o_neg in o_sample_list:
        out_pairs.append((s, o_neg))
        
    return out_pairs

In [28]:
def eval_pair(pair, sample_rate = 10):
    
    pair_negs = generate_negatives(pair, sample_rate=sample_rate)
    
    agatha_queries = [pair] + pair_negs
    
    pair_labels = [1] + [0]*len(pair_negs)
    
    scores = model.predict_from_terms(agatha_queries)
    
    res_list = sorted(
        list(zip(scores, pair_labels)),
        key=lambda x: x[0],
        reverse=True
    )
    
    rank = None
    for i, (score, lbl) in enumerate(res_list):
        if lbl == 1:
            rank = i
            break
    
    return {
        'scores': res_list,
        'pos_rank': rank + 1
    }

In [29]:
def evaluate_llm_resp(llm_resp_str):
    expl_parts_list = []
    cleaned_text = llm_resp_str.replace('\n', '').replace('*', '')
    expl_sr_out = sr_h.ProcessList_parallel([cleaned_text])

    total_agatha_score = 0
    count_agatha_score = 0

    for s_id, sent_data in expl_sr_out.items():
        sent_text = sent_data['sent_text']

        status = 'ok'
        bad_predicates = []
        good_predicates = []
        agatha_score_path = []

        for rel in sent_data['relations']:
            rel_subj = rel['subj_text']
            v = rel['verb']
            rel_obj = rel['obj_text']

            pred = [rel_subj, rel_obj]

            rel_pair = tuple(
                sorted(
                    [
                        rel['subj_id'],
                        rel['obj_id']
                    ]
                )
            )
            if rel_pair in pred_cui_pairs_set:
                good_predicates.append(pred)
            if rel_pair not in pred_cui_pairs_set:
                subj_id = f"m:{rel['subj_id'].lower()}"
                obj_id = f"m:{rel['obj_id'].lower()}"
                if subj_id in model.graph and obj_id in model.graph:
                    agatha_score = model.predict_from_terms([[subj_id, obj_id]])[0]
                    agatha_rank = eval_pair([subj_id, obj_id], sample_rate=20)['pos_rank']
                    pred.append(agatha_rank)
                    if agatha_rank > 1:
                        status = 'REWORK'
                        bad_predicates.append(pred)
                        agatha_score_path.append(agatha_score)
                        print("Agatha score for predicate: {}".format(agatha_score))
                    else:
                        good_predicates.append(pred)
                        agatha_score_path.append(agatha_score)

                    total_agatha_score += agatha_score
                    count_agatha_score += 1

        print('Sentence:\n', sent_text)
        print('Status:\n', status)
        if status != 'ok':
            print('Reason:')
            for p in bad_predicates:
                print('\t', p, 'not in AGATHA KB')
        else:
            if len(good_predicates):
                print('Recognized:')
            for p in good_predicates:
                print('\t', p, 'in AGATHA KB')

        print('\n-----\n')

        expl_parts_list.append(
            {
                'sent_text': sent_text,
                'status': status,
                'bad_predicates': bad_predicates,
                'good_predicates': good_predicates,
                'agatha_score': agatha_score_path
            }
        )

    if count_agatha_score > 0:
        average_agatha_score = total_agatha_score / count_agatha_score
        print("Average Agatha score: {}".format(average_agatha_score))
    else:
        print("No Agatha scores available to calculate average.")

    return expl_parts_list

In [38]:
%%time
eval_result = evaluate_llm_resp(llm_resp_path_1)
eval_result

Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...
Sentence:
 Based on the provided abstracts, an indirect relationship between GABBR1 and Baclofen can be described as follows:1. 
Status:
 ok

-----

Sentence:
 GABBR1 and OCD: The first abstract discusses the potential role of DNA methylation in the gamma-aminobutyric acid B receptor 1 (GABBR1) gene in the context of obsessive-compulsive disorder (OCD). 
Status:
 ok
Recognized:
	 ['DNA methylation', 'gene'] in AGATHA KB

-----

Agatha score for predicate: 0.7814140319824219
Sentence:
 The study suggests preliminary support for differential methylation of GABBR1 in individuals with OCD, which may correlate with the severity of the disorder and treatment outcomes. 
Status:
 REWORK
Reason:
	 ['individuals', 'GABBR1', 8] not in AGATHA KB

-----

Sentence:
 This indicates that GABBR1 could be involved in the pathophysiology of OCD, potentially affecting the GABAergic system.2. 
Status:
 ok
Recognized:
	 ['GA

[{'sent_text': 'Based on the provided abstracts, an indirect relationship between GABBR1 and Baclofen can be described as follows:1. ',
  'status': 'ok',
  'bad_predicates': [],
  'good_predicates': [],
  'agatha_score': []},
 {'sent_text': 'GABBR1 and OCD: The first abstract discusses the potential role of DNA methylation in the gamma-aminobutyric acid B receptor 1 (GABBR1) gene in the context of obsessive-compulsive disorder (OCD). ',
  'status': 'ok',
  'bad_predicates': [],
  'good_predicates': [['DNA methylation', 'gene']],
  'agatha_score': []},
 {'sent_text': 'The study suggests preliminary support for differential methylation of GABBR1 in individuals with OCD, which may correlate with the severity of the disorder and treatment outcomes. ',
  'status': 'REWORK',
  'bad_predicates': [['individuals', 'GABBR1', 8]],
  'good_predicates': [['OCD', 'individuals']],
  'agatha_score': [0.7814140319824219]},
 {'sent_text': 'This indicates that GABBR1 could be involved in the pathophysiol

### 3.1. List of Rework Sentences

In [39]:
expl_parts_df = pd.DataFrame(eval_result).reset_index()
expl_parts_df['reworked_part'] = None
expl_parts_df

,index,sent_text,status,bad_predicates,good_predicates,agatha_score,reworked_part
0,0,"Based on the provided abstracts, an indirect relationship between GABBR1 and Baclofen can be described as follows:1.",ok,[],[],[],None
1,1,GABBR1 and OCD: The first abstract discusses the potential role of DNA methylation in the gamma-aminobutyric acid B receptor 1 (GABBR1) gene in the context of obsessive-compulsive disorder (OCD).,ok,[],"[[DNA methylation, gene]]",[],None
2,2,"The study suggests preliminary support for differential methylation of GABBR1 in individuals with OCD, which may correlate with the severity of the disorder and treatment outcomes.",REWORK,"[[individuals, GABBR1, 8]]","[[OCD, individuals]]",[0.7814140319824219],None
3,3,"This indicates that GABBR1 could be involved in the pathophysiology of OCD, potentially affecting the GABAergic system.2.",ok,[],"[[GABBR1, OCD]]",[],None
4,4,OCD and LUTS: The second abstract highlights an association between OCD and lower urinary tract symptoms (LUTS) in women.,ok,[],"[[OCD, women], [lower urinary tract symptoms, women]]",[],None
5,5,"It suggests that women with OCD are more likely to experience voiding dysfunction and urodynamic abnormalities, such as bladder outlet obstruction.",ok,[],"[[voiding dysfunction, women], [OCD, women]]",[],None
6,6,This implies a possible link between neuropsychiatric conditions like OCD and urinary symptoms.3.,ok,[],[],[],None
7,7,"Baclofen and LUTS: The third abstract evaluates the use of Baclofen, a GABA_B receptor agonist, in combination with antimuscarinics to treat women with overactive bladder (OAB) and abnormal voiding patterns.",REWORK,"[[antimuscarinics, abnormal voiding, 11], [Baclofen, abnormal voiding, 9], [GABA_B receptor agonist, abnormal voiding, 11]]","[[antimuscarinics, overactive bladder], [Baclofen, overactive bladder], [GABA_B receptor agonist, overactive bladder, 1], [abnormal voiding, women], [overactive bladder, women], [antimuscarinics, women], [Baclofen, women], [GABA_B receptor agonist, women, 1]]","[0.4506675601005554, 0.494037002325058, 0.21089904010295868, 0.5443702340126038, 0.7128509879112244]",None
8,8,"The study finds that Baclofen can reduce voiding difficulties when added to antimuscarinic treatment, suggesting its efficacy in managing urinary symptoms.### Indirect Relationship:- GABBR1 and Baclofen: GABBR1 encodes the GABA_B receptor, which is the target of Baclofen.",ok,[],"[[Baclofen, voiding difficulties], [GABA_B receptor, Baclofen]]",[],None
9,9,"The first abstract indicates that GABBR1 may be epigenetically altered in OCD, potentially affecting GABAergic signaling.",ok,[],"[[GABBR1, OCD]]",[],None


In [40]:
expl_parts_enum_dict = dict(enumerate(eval_result))
rework_sentences = []

for key, value in expl_parts_enum_dict.items():
    if value['status'] == 'REWORK':  
        rework_sentences.append(value['sent_text']) 
print(rework_sentences)
len(rework_sentences)

['The study suggests preliminary support for differential methylation of GABBR1 in individuals with OCD, which may correlate with the severity of the disorder and treatment outcomes. ', 'Baclofen and LUTS: The third abstract evaluates the use of Baclofen, a GABA_B receptor agonist, in combination with antimuscarinics to treat women with overactive bladder (OAB) and abnormal voiding patterns. ', 'The second abstract connects OCD with LUTS, and the third abstract shows that Baclofen, a GABA_B receptor agonist, can alleviate voiding difficulties in OAB.- Connecting the Dots: Although the studies do not directly link GABBR1 and Baclofen, they collectively suggest an indirect relationship. ', 'Baclofen, by targeting the GABA_B receptor, could help manage these urinary symptoms, indicating a therapeutic pathway that involves the GABA_B receptor system.In summary, the indirect relationship between GABBR1 and Baclofen is mediated through their roles in OCD and LUTS. ', 'Alterations in GABBR1 m

5

### 3.2. Find_top_k doc

In [38]:
model_sbert = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def find_top_documents_for_queries(queries, path_context_dict, top_k=1, return_scores=False):
    # Flatten the contexts into a list of (pmid, context) tuples
    abstract_texts = []
    pmid_list = []
    for edges, pmid_dict in path_context_dict.items():
        for pmid, context in pmid_dict.items():
            abstract_texts.append(context)
            pmid_list.append(pmid)
    
    context_embeddings = model_sbert.encode(abstract_texts, convert_to_tensor=True)
    results = []

    # Process each query
    for query in queries:
        query_embedding = model_sbert.encode(query, convert_to_tensor=True)
        search_results = util.semantic_search(query_embedding, context_embeddings, top_k=top_k)
        top_results = search_results[0]
        
        if return_scores:
            top_pmids_scores = [(pmid_list[result['corpus_id']], result['score']) for result in top_results]
            results.append(top_pmids_scores)
        else:
            top_pmids = [pmid_list[result['corpus_id']] for result in top_results]
            results.append(top_pmids)

    return results

In [42]:
%%time
top_documents = find_top_documents_for_queries(rework_sentences, path_context_dict, top_k=1)
top_documents

CPU times: user 1.84 s, sys: 1.89 s, total: 3.74 s
Wall time: 4.87 s


[['27047397'], ['22905457'], ['34787920'], ['34787920'], ['27047397']]

## 4. Update Prompt based on prior rework sentences:

In [30]:
source_cui = 'GABBR1 gene'  # 'C1414904'
target_cui = 'Baclofen'     # 'C0004609'
llm_prompt_template = '''How would you describe an indirect relationship between {source} and {target} given the following scientific abstracts as context? 

Please note that the following sentence from the last response is incorrect: 
"{rework_sentences}"

Context from scientific literature: 
'''

llm_fix_prompt_str = '\n\n'.join(path_context_abstracts)
llm_fix_prompt_combo = llm_prompt_template.format(source=source_cui, target=target_cui, rework_sentences=rework_sentences) +  '\n\n' + llm_fix_prompt_str
llm_fix_prompt_combo

'How would you describe an indirect relationship between GABBR1 gene and Baclofen given the following scientific abstracts as context? \n\nPlease note that the following sentence from the last response is incorrect: \n"[\'The third abstract demonstrates that Baclofen can inhibit various sensory nerve fibers, including those expressing VGLUT3, through its action on GABAB receptors.In summary, the indirect relationship between the GABBR1 gene and Baclofen can be described as follows: Variants in the GABBR1 gene, which encodes a subunit of the GABAB receptor, influence the risk of alcoholism and potentially the response to Baclofen treatment. \']"\n\nContext from scientific literature: \n\n\nGABBR1 and SLC6A1, Two Genes Involved in Modulation of GABA Synaptic Transmission, Influence Risk for Alcoholism: Results from Three Ethnically Diverse Populations. Animal and human studies indicate that GABBR1, encoding the GABAB1 receptor subunit, and SLC6A1, encoding the neuronal gamma-aminobutyric

In [21]:
agatha_sent_db_path = (
    '/lustre/acslab/shared/Agatha_shared/2021_11_22_full/sentences.sqlite3'
)
sents_db = Sqlite3LookupTable(agatha_sent_db_path)

In [20]:
def retrieve_abstracts_from_pmids(pmids):
    abstracts = []
    
    for pmid in pmids:
        abstr_text = text_util.get_abstr_text(pmid, sents_db)
        abstracts.append(abstr_text)
    
    return abstracts 

## 5. Automated Feedback Loop:

In [81]:
def feedback_loop(top_20_df, path_context_dict, sents_db, max_iterations=3):
    print('\n-------------------------Step 1: prepare ----------------------------\n')
    # Step 1: Get Path
    path_153_context_pmids = top_20_df.loc[186]['context_pmids']
    path_context_dict = text_util.get_path_context(path_153_context_pmids, sents_db)
    pmid_list = [pubid for path in path_context_dict.values() for pubid in path.keys()]
    print(f"Here is the choosen pmids for this path {pmid_list}")
    path_context_abstracts = semantic_search_util.retrieve_context_path(path_context_dict)
    existing_context_pmids = set(path_context_abstracts)
    # print(f"Here is the Existing pmids for this path {existing_context_pmids}")

    print('\n------------------step 2 Prepare prompt + context -----------------------------------\n')

    # Step 2: Prepare initial prompt
    llm_prompt_template = 'How would you describe an indirect relationship between COMT and Tolcapone given the following scientific abstracts as context?'
    llm_fix_prompt_str = '\n\n'.join(path_context_abstracts)
    llm_fix_prompt_combo = llm_prompt_template + '\n\n' + llm_fix_prompt_str
    print(llm_fix_prompt_combo)

    for iteration in tqdm(range(max_iterations), desc="Iterations"):
        print(f"Iteration {iteration + 1}") 

        print('\n------------------step 3: Response generated from LLM -----------------------------------\n')   
        # Step 3: Get Response from LLM
        try:
            llm_resp_path = oai_get_response(llm_fix_prompt_combo, temp=1e-19, top_p=1e-9, seed=1234)
            print(f"Response from LLM at iteration {iteration + 1}:\n{llm_resp_path}")
        except Exception as e:
            print(f"Error getting response from LLM at iteration {iteration + 1}: {e}")
            time.sleep(20)  # Wait for 20 seconds before retrying
            continue

        print('\n------------------step 4: Start Evaluation -----------------------------------\n')
        # Step 4: Evaluate Response
        eval_result_path = evaluate_llm_resp(llm_resp_path)
        print('\n------------------step 4: Evaluation by Agatha is Done -----------------------------------\n')
        
        # Extract rework sentences
        expl_parts_enum_dict = dict(enumerate(eval_result_path))
        rework_sentences = []
        for key, value in expl_parts_enum_dict.items():
            if value['status'] == 'REWORK':  
                rework_sentences.append(value['sent_text'])

        # Print rework sentences for the current iteration
        if rework_sentences:
            print(f"Rework sentences at iteration {iteration + 1}: {rework_sentences}")
        else:
            print("There is no Rework Sentence, Satisfactory response achieved.")
            return llm_resp_path

        print('\n------------------step 5 Update Context-----------------------------------\n')
        # Step 5: Update Context
        new_context_pmids = []
        for rework_sent in rework_sentences:
            top_pmid_pop = find_top_documents_for_queries(rework_sentences, path_context_dict, top_k=1)
            print(f"Here is the top_1 closest pmids for this sentence {top_pmid_pop}")
            # Filter out PMIDs already in the context
            if len(existing_context_pmids) < 3:
                print("There is no pmid to replace, stop path, Satisfactory response achieved.")
                return llm_resp_path
            else:
                # Ensure all PMIDs are hashable types
                top_pmid_pop = [tuple(pmid) if isinstance(pmid, list) else pmid for pmid in top_pmid_pop]
                
                # Convert existing_context_pmids to tuples if necessary
                existing_context_pmids = {tuple(pmid) if isinstance(pmid, list) else pmid for pmid in existing_context_pmids}
        
                new_pmids = [pmid for pmid in top_pmid_pop if pmid not in existing_context_pmids]
                new_context_pmids.extend(new_pmids)  # Collect new PMIDs
        
        # Retrieve abstracts for new PMIDs
        new_context_abstracts = retrieve_abstracts_from_pmids(new_context_pmids)
        
        # Add the new PMIDs to the existing context set
        existing_context_pmids.update(new_context_pmids)
        
        # Log the new contexts
        print(f"New contexts at iteration {iteration + 1}: {new_context_abstracts}")
        
        # Add new contexts to the prompt
        if new_context_abstracts:
            new_context_str = '\n\n'.join(new_context_abstracts)
            llm_fix_prompt_combo += '\n\n' + new_context_str
        
        print('\n------------------Step 6 Update Prompt based on Memory of Rework Sentence-----------------------------------\n')
        # Prepare prompt for next iteration if needed
        llm_prompt_template = '''How would you describe an indirect relationship between COMT and Tolcapone given the following scientific abstracts as context? 
        
        Please note that the following sentence from the last response is incorrect: 
        "{rework_sentence}"
        
        Context from scientific literature: 
        '''
        llm_fix_prompt_combo = llm_prompt_template.format(rework_sentence=rework_sentences[0]) + '\n\n' + llm_fix_prompt_str
        
        print("Maximum iterations reached without satisfactory response.")
        return llm_resp_path

In [82]:
%%time
response = feedback_loop(top_20_df,path_context_dict, sents_db, max_iterations=3)


-------------------------Step 1: prepare ----------------------------

Here is the choosen pmids for this path ['20667552', '27283209', '25019647', '20934466']

------------------step 2 Prepare prompt + context -----------------------------------

How would you describe an indirect relationship between COMT and Tolcapone given the following scientific abstracts as context?

Catechol-o-methyltransferase gene modulation on suicidal behavior and personality traits: review, meta-analysis and association study. Suicide is one of the leading causes of death among young adults. Both genetic and personality factors plausibly have a role on suicidal behavior. We focused on the catechol-O-methyltransferase gene (COMT) and we performed: a review of studies investigating the association between COMT and both suicidal behavior and personality; a meta-analysis of studies investigating the association between suicidal behavior and COMT rs4680 polymorphism; an association study investigating the link

Iterations:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration 1

------------------step 3: Response generated from LLM -----------------------------------

Response from LLM at iteration 1:
An indirect relationship between catechol-O-methyltransferase (COMT) and tolcapone can be inferred from the provided abstracts, particularly focusing on their roles in cognitive function and behavior.

1. **COMT and Personality Traits**: The first abstract discusses the role of the COMT gene in modulating personality traits and its potential, though not direct, association with suicidal behavior. COMT is an enzyme that degrades catecholamines such as dopamine, epinephrine, and norepinephrine, which are crucial for various brain functions, including mood regulation and cognitive processes.

2. **Tolcapone as a COMT Inhibitor**: The fourth abstract examines the effects of tolcapone, a COMT inhibitor, on cognitive performance in rats. The study found that tolcapone did not affect sub-optimal performance in the 5-choice serial reaction time task (5-CSRTT

Iterations:   0%|          | 0/3 [00:37<?, ?it/s]

Agatha score for predicate: 0.6207326054573059
Sentence:
 While atomoxetine showed some effects on impulsive behavior, tolcapone did not demonstrate similar efficacy in the animal model.4. 
Status:
 REWORK
Reason:
	 ['tolcapone', 'impulsive behavior', 2] not in AGATHA KB

-----

Sentence:
 Indirect Relationship: The indirect relationship between COMT and tolcapone can be summarized as follows:   - COMT's Role: COMT is involved in the breakdown of dopamine and other catecholamines, influencing cognitive functions and personality traits. 
Status:
 ok

-----

Sentence:
 - Tolcapone's Mechanism: Tolcapone inhibits COMT, thereby increasing the availability of dopamine and other catecholamines in the brain. 
Status:
 ok
Recognized:
	 ['brain', 'catecholamines'] in AGATHA KB
	 ['brain', 'dopamine'] in AGATHA KB
	 ['Tolcapone', 'COMT'] in AGATHA KB

-----

Sentence:
 - Behavioral and Cognitive Outcomes: Despite tolcapone's biochemical action on COMT, it did not translate into improved cognitiv

-----------------------------------------------------------------------